In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Carregar o dataset
file_path = './Datasets/20241016 Gym Members Exercise Dataset/gym_members_exercise_tracking.csv'
data = pd.read_csv(file_path)

In [4]:
# Para identificar atributos que podem não contribuir significativamente para o modelo, 
# vamos usar uma técnica de Seleção de Atributos. Aqui, utilizaremos a Correlação entre variáveis para identificar 
# aquelas que são menos correlacionadas com a variável-alvo 
# (neste caso, podemos definir como a coluna Calories_Burned)

In [5]:
# Converter a coluna categórica 'Gender' em valores numéricos
data['Gender'] = data['Gender'].map({'Male': 1, 'Female': 0})



In [6]:
# Converter a coluna categórica 'Workout_Type' em valores numéricos usando codificação de variável fictícia (one-hot encoding)
data_encoded = pd.get_dummies(data, columns=['Workout_Type'])

# Recalcular a correlação novamente após tratar todas as colunas categóricas
correlation = data_encoded.corr()
correlation_with_target = correlation['Calories_Burned'].sort_values(ascending=False)
correlation_with_target

Calories_Burned                  1.000000
Session_Duration (hours)         0.908140
Experience_Level                 0.694129
Workout_Frequency (days/week)    0.576150
Water_Intake (liters)            0.356931
Avg_BPM                          0.339659
Gender                           0.150646
Weight (kg)                      0.095443
Height (m)                       0.086348
BMI                              0.059761
Workout_Type_HIIT                0.040550
Resting_BPM                      0.016518
Workout_Type_Strength            0.011629
Max_BPM                          0.002090
Workout_Type_Yoga               -0.004678
Workout_Type_Cardio             -0.045726
Age                             -0.154679
Fat_Percentage                  -0.597615
Name: Calories_Burned, dtype: float64

In [7]:
# Os atributos com menor correlação com a variável Calories_Burned são:

# Max_BPM (0.002)
# Resting_BPM (0.017)
# BMI (0.060)
# Height (m) (0.086)
# Esses atributos podem ter uma contribuição reduzida para a variável-alvo Calories_Burned. 
# Removendo esses atributos e re-treinando o algoritmo de agrupamento para avaliar o impacto.

In [8]:
# Preparar os dados para clustering, removendo colunas categóricas e a variável alvo para facilitar a análise
data_numerical = data.select_dtypes(include=['float64', 'int64']).drop(columns=['Calories_Burned'])

In [9]:
# Verificar valores ausentes ou inconsistentes no dataset
data_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)  # Substituir valores infinitos por NaN
data_encoded.dropna(inplace=True)  # Remover linhas com NaN para garantir integridade dos dados

In [11]:
# Reconstruir 'data_reduced' com as colunas relevantes, removendo atributos de baixa correlação
data_reduced = data_encoded.drop(columns=[
    'Calories_Burned', 'Max_BPM', 'Resting_BPM', 'BMI', 'Height (m)', 
    'Workout_Type_Yoga', 'Workout_Type_Cardio'
])

In [12]:
# Redefinir o escalador e normalizar os dados reduzidos
scaler = StandardScaler()
data_reduced_scaled = scaler.fit_transform(data_reduced)

In [13]:
# Configurar e executar o modelo K-Means
kmeans_reduced = KMeans(n_clusters=3, random_state=42)
clusters_reduced = kmeans_reduced.fit_predict(data_reduced_scaled)

C:\Users\acsfarias\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [14]:
# Avaliar o modelo com o índice de silhueta
silhouette_reduced = silhouette_score(data_reduced_scaled, clusters_reduced)
silhouette_reduced

0.2183534440760568

In [ ]:
# O índice de silhueta para o modelo K-Means com o dataset reduzido é 0.218, 
# o que representa uma leve melhoria em comparação com o índice original de 0.208.

# Esse resultado indica que a remoção dos atributos de baixa correlação contribuiu 
# para uma qualidade de agrupamento ligeiramente melhor.